In [1]:
%reload_ext autoreload
%autoreload 2
from aldiscore.prediction.extractor import FeatureExtractor
from Bio.SeqRecord import SeqRecord
import os
from pathlib import Path
from Bio import SeqIO
from aldiscore import ROOT, get_from_config
import numpy as np
import pandas as pd

Trials:
- tc_base vs tc_dist_abs vs tc_dist_scaled vs tc_dist_log
- tc_base + (tc_dist_abs vs tc_dist_scaled vs tc_dist_log)
- tc_base vs tc_50 vs tc_100

In [16]:
TEST_FILE_AA = Path("data/aa.fasta")
TEST_FILE_DNA = Path("data/dna.fasta")

sequences = list(SeqIO.parse(TEST_FILE_DNA, "fasta"))
print("k =", len(sequences))
dummy = sequences

# print(len(dummy))
extractor = FeatureExtractor(dummy, track_perf=True)
# feat_df = extractor.compute()
extractor._init_cache()
# feats = extractor._get_ent_features()
feats = extractor.compute()
# feats.info()
feats
# print(
#     feats.loc[
#         :, feats.columns.str.contains("tc") & feats.columns.str.contains("mean:")
#     ].T
# )

k = 4


,is_dna,num_seqs,min:seq_length,max:seq_length,mean:seq_length,std:seq_length,p5:seq_length,p10:seq_length,p25:seq_length,p50:seq_length,...,max:tc_200_max,mean:tc_200_max,std:tc_200_max,p5:tc_200_max,p10:tc_200_max,p25:tc_200_max,p50:tc_200_max,p75:tc_200_max,p90:tc_200_max,p95:tc_200_max
0,True,4,201,237,218.25,13.141062,202.8,204.6,210.0,217.5,...,0.335938,0.247697,0.057245,0.187154,0.19249,0.208498,0.236515,0.275714,0.311848,0.323893


In [ ]:
perf_dir = ROOT / "logs" / "perf"
perf_file = sorted(os.listdir(perf_dir))[-1]
print(perf_file)
perf_df = pd.read_parquet(perf_dir / perf_file)
perf_df.groupby("source").max()

2025-08-20_22-38-20.parquet


_init_cache                            135.772175
_data_type                               0.000171
_num_sequences                           0.000223
_sequence_length                         0.025980
_lower_bound_gap_percentage              0.028097
_sequence_js_divergence                  0.149940
_ent_randomness                          3.132074
_transitive_consistency                  0.663932
_transitive_consistency_dist             0.717448
_transitive_consistency_dist_scaled      0.638519
_transitive_consistency_dist_log         0.675452
_transitive_consistency_50               0.103941
_transitive_consistency_100              0.218868
_transitive_consistency_200              0.392059
dtype: float64

In [2]:
from tqdm import tqdm

source = "treebase_v1"
data_dir = Path("/hits/fast/cme/bodynems/data/paper") / source
datasets = list(
    filter(lambda name: os.path.isdir(data_dir / name), os.listdir(data_dir))
)

feats_dict = {}
for i, dataset in tqdm(enumerate(datasets)):
    # if dataset != "25479_0.phy":
    #     continue
    seqs = list(SeqIO.parse(data_dir / dataset / "sequences.fasta", format="fasta"))
    feats = FeatureExtractor(seqs).compute()
    feats_dict[(source, dataset)] = feats
    if i >= 24:
        break
feat_df = pd.concat(feats_dict.values(), axis=0, ignore_index=True)
feat_df.index = pd.MultiIndex.from_tuples(
    feats_dict.keys(), names=["source", "dataset"]
)

# feat_df.info()

11it [00:06,  1.63it/s]


KeyboardInterrupt: 

In [ ]:
import parasail
from Bio.Seq import Seq
from Bio import SeqIO
import itertools
import random
from aldiscore.scoring.encoding import encode_positions
from aldiscore.enums.enums import PositionalEncodingEnum as PSE
from collections import defaultdict

seq_path = Path("/hits/fast/cme/bodynems/data/paper/prefab4/1cvl_1tca/sequences.fasta")
sequences = list(SeqIO.parse(seq_path, format="fasta"))
print(len(sequences))
print(sequences)
GAP_CHAR = "-"
# sequences = [
#     SeqRecord(Seq("ACCCA")),
#     SeqRecord(Seq("ACAGTAAACCA")),
#     SeqRecord(Seq("ACGGTACATTA")),
#     SeqRecord(Seq("TTCTTAG")),
#     SeqRecord(Seq("GCTACTATTA")),
#     SeqRecord(Seq("TTAATGGGGGGA")),
#     SeqRecord(Seq("ATTTCTA")),
#     SeqRecord(Seq("ATTTATTAAACCA")),
#     SeqRecord(Seq("ATTTGTTACATTA")),
#     SeqRecord(Seq("TTTTCTTAG")),
#     SeqRecord(Seq("GTTTTTCTATTA")),
#     SeqRecord(Seq("TTTTATTGGGGGGA")),
# ]
extractor = FeatureExtractor(sequences)
extractor._init_cache()
out = extractor.compute()
print(len(out.columns))
print(len(out.columns[out.columns.str.contains("tc_")]))
for i, col in enumerate(out.columns):
    print(col, end=" | ")
    if (i + 1) % 6 == 0:
        print()
# idx_map = extractor._cache[extractor._PSA_INDEX_MAP]
# for idx_a in idx_map:
#     print(idx_map[idx_a])

# print(groups)
# for idx_a in aligns:
#     print(idx_a)
#     for idx_b in aligns[idx_a]:
#         print("  " + str(idx_b))

50
[SeqRecord(seq=Seq('ADTYAATRYPVILVHGLAGTDKFANVVDYWYGIQSDLQSHGAKVYVANLSGFQS...QGV'), id='1cvl', name='1cvl', description='1cvl', dbxrefs=[]), SeqRecord(seq=Seq('SSYAKTKYPILLAHGMAGFSAVGPLQYWNGITEDLVGNGANVFVAQQASFNSSE...QGL'), id='19|1cvl|gi|4768272', name='19|1cvl|gi|4768272', description='19|1cvl|gi|4768272', dbxrefs=[]), SeqRecord(seq=Seq('NYTKTKYPIVLVHGVTGFNTIGGLVNYFHTIPWNLERDGARVHVASVAAFNDSE...KGV'), id='24|1cvl|gi|2072017', name='24|1cvl|gi|2072017', description='24|1cvl|gi|2072017', dbxrefs=[]), SeqRecord(seq=Seq('ATDYTRTRYPIVLSHGLFGFKSVGPVDYWHAIVPALEKDGAKVFATSQSPVNSN...VGL'), id='11|1cvl|gi|9951080', name='11|1cvl|gi|9951080', description='11|1cvl|gi|9951080', dbxrefs=[]), SeqRecord(seq=Seq('AGEGHPVVLLHGSGPGATGWSNYSDNIEALSRHFHVYAVDLPGWGDSDPADFAT...FLK'), id='34|1cvl|gi|3059193', name='34|1cvl|gi|3059193', description='34|1cvl|gi|3059193', dbxrefs=[]), SeqRecord(seq=Seq('ATRYPLVLVPGMLGFVRMVLYPYWFGIVPALRKGGAQVFPVQVSPLHSSEVRGE...AGL'), id='39|1cvl|gi|24986618', name='39|1cvl|gi|24